In [ ]:
# Install dependencies
!pip install -q gradio faiss-cpu sentence-transformers pandas requests

# ---------- Imports ----------
import gradio as gr
import faiss
import numpy as np
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer

# ---------- NVIDIA API ----------
NVIDIA_API_KEY = "NVIDIA_API_KEY" 
NVIDIA_CHAT_URL = "CHAT_API_ENDPOINT" 
headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Content-Type": "application/json"
}

def nvidia_llm(query, context=""):
    try:
        payload = {
            "model": "meta/llama-3.1-8b-instruct",
            "messages": [
                {"role": "system", "content": "You are a helpful eCommerce assistant."},
                {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
            ],
            "temperature": 0.3
        }
        resp = requests.post(NVIDIA_CHAT_URL, headers=headers, json=payload)
        return resp.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"⚠️ LLM call failed: {e}"

# ---------- Embeddings & FAISS ----------
embedder = SentenceTransformer("all-MiniLM-L6-v2")
dimension = 384
index = faiss.IndexFlatL2(dimension)
products = []  # Stores product descriptions

# ---------- Auto-update Knowledge Base ----------
def update_products(csv_file):
    global products, index
    try:
        df = pd.read_csv(csv_file.name)
        product_texts = (df["product_name"].astype(str) + " " +
                         df["description"].astype(str) + " " +
                         df["category"].astype(str)).tolist()
        embeddings = embedder.encode(product_texts)
        index.reset()
        index.add(np.array(embeddings, dtype=np.float32))
        products = product_texts
        return f"✅ Loaded {len(products)} products into knowledge base."
    except Exception as e:
        return f"⚠️ Failed to load products: {e}"

# ---------- RAG Query ----------
def rag_query(query):
    if len(products) == 0:
        return "⚠️ No product data loaded. Please upload product CSV."

    vec = embedder.encode([query])
    D, I = index.search(np.array(vec, dtype=np.float32), k=3)
    if len(I[0]) == 0 or I[0][0] == -1:
        return "⚠️ No relevant products found."

    context = "\n".join([products[i] for i in I[0]])
    answer = nvidia_llm(query, context)
    return answer

# ---------- Gradio UI ----------
with gr.Blocks() as demo:
    gr.Markdown("## 🛒 RAG eCommerce Assistant (NVIDIA + FAISS)")

    with gr.Tab("📂 Upload Products"):
        csv_upload = gr.File(label="Upload Product CSV", file_types=[".csv"])
        upload_status = gr.Textbox(label="Status")
        csv_upload.upload(update_products, csv_upload, upload_status)

    with gr.Tab("💬 Chatbot"):
        query = gr.Textbox(label="Ask about products, recommendations, or suggestions")
        answer = gr.Textbox(label="LLM Response", lines=15)
        ask_btn = gr.Button("Ask")
        ask_btn.click(rag_query, query, answer)

    with gr.Tab("📊 Embeddings Viewer"):
        emb_inp = gr.Textbox(label="Enter text to see embeddings")
        emb_btn = gr.Button("Generate Embeddings")
        emb_out = gr.Textbox(label="Embedding Vector", lines=15)
        def show_embeddings(text):
            vec = embedder.encode([text])[0]
            return f"🔢 Embedding vector (length {len(vec)}):\n\n{vec}"
        emb_btn.click(show_embeddings, emb_inp, emb_out)

demo.launch(inline=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://008f56c481bf29cdeb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
